In [1]:
import nest_asyncio
nest_asyncio.apply()
from telethon import TelegramClient
from keys import api_id2, api_hash2, my_username2, my_username
import pickle

import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, TrainingArguments, DataCollatorForLanguageModeling
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import pickle
import string
import matplotlib.pyplot as plt
import re
from collections import Counter
import time
import random

In [20]:
import nest_asyncio
nest_asyncio.apply()
from telethon import TelegramClient
# from keys import api_id2, api_hash2, my_username2
import pickle


In [25]:
api_id2 = 12636152
api_hash2 = "4cea86eb7174aa3748b5d6d2f2797925"
my_username = "cherryquinnlg"

In [ ]:
model_custom = GPT2LMHeadModel.from_pretrained("/Users/bayesiandater/ex_bot/resources")

In [11]:
msg_from_ex = pickle.load(open("/Users/bayesiandater/ex_bot/resources/msg_from_ex", "rb"))
msg_from_ex = [m.lower() for m in msg_from_ex if m.strip() != ""]

rng = np.random.default_rng(713)
train_idx = rng.integers(low=0, high=len(msg_from_ex), size=int(0.8 * len(msg_from_ex)))
valid_idx = [i for i in range(len(msg_from_ex)) if i not in train_idx]
train_ls, valid_ls = np.array(msg_from_ex)[train_idx], np.array(msg_from_ex)[valid_idx]

added_tokens = ["coz", "noice", "lmaooo", "ooh", "uhh", "prolly", "yesss", "nvm", "ooof", "thingy", '👀',]

class MSG(Dataset):  
    def __init__(self, ls_stc, truncate=False, gpt2_type="gpt2", max_length=1024, added_tokens=None):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
        self.tokenizer.add_special_tokens(special_tokens)
        if added_tokens is not None:
            self.tokenizer.add_tokens(added_tokens)
        self.msgs = []
        max_len = 0
        
        for m in ls_stc:
            m = torch.tensor(
                self.tokenizer.encode(self.tokenizer.bos_token + f"{m[:max_length]}" + self.tokenizer.eos_token, \
                                      max_length=15, padding="max_length", truncation=True))
            max_len = max(len(m), max_len)

            self.msgs.append(m)
        
        if truncate:
            self.msgs = self.msgs[:20000]
            
        self.msgs_count = len(self.msgs)
        
    def __len__(self):
        return self.msgs_count

    def __getitem__(self, item):
        return self.msgs[item]
    

In [12]:
ds_custom_train = MSG(train_ls, truncate=True, gpt2_type="gpt2", added_tokens=added_tokens)  
ds_custom_valid = MSG(valid_ls, truncate=True, gpt2_type="gpt2", added_tokens=added_tokens)  


In [23]:
def my_generator():
    max_length = 100
    num_beams=20
    num_return_sequences= 1
    early_stopping=False 
    no_repeat_ngram_size=2
    do_sample = True

    while True:
        j = random.randint(0, len(ds_custom_valid) - 1)
        m = list(ds_custom_valid)[j]
        ops = model_custom.generate(m[:3].unsqueeze(0), 
                                    max_length = max_length,
                                    do_sample=do_sample,
                                    num_return_sequences=num_return_sequences,
                                    no_repeat_ngram_size=no_repeat_ngram_size,
                                    early_stopping=False)

        beam = ops[0]
        o = ds_custom_train.tokenizer.decode(beam, skip_special_tokens=True)
        o = re.sub(r"[ +|\n]", " ", o).strip("\n ")
        o = re.sub(r" +", " ", o).strip("\n ")
        if o not in ["",  " "]:
            break
    
    return o



In [27]:
async with TelegramClient('new2', api_id2, api_hash2) as client:
    while True:
        m = my_generator()
        client.loop.run_until_complete(client.send_message(my_username, m))
        time.sleep(random.randint(10, 20))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

IndexError: list index out of range